In [5]:
# ===================== Batch evaluator (with inverse-depth support) =====================
import os, json
import numpy as np
import pandas as pd
import cv2

# -------------------- 公共函数 --------------------
GT_MIN_VALID = 1e-3  # 防止极小正数导致 AbsRel 爆炸

def list_stems(folder):
    files = [f for f in os.listdir(folder) if f.endswith(".npy")]
    return {os.path.splitext(f)[0]: os.path.join(folder, f) for f in files}

def list_mask_index(folder):
    """把 *_depth_mask.npy 映射到 GT 的 *_depth.npy 的 stem，便于和 GT 配对。"""
    if not folder or (isinstance(folder, str) and not os.path.isdir(folder)):
        return {}
    out = {}
    for f in os.listdir(folder):
        if not f.endswith(".npy"):
            continue
        stem = os.path.splitext(f)[0]
        if stem.endswith("_depth_mask"):
            gt_stem = stem[:-len("_depth_mask")] + "_depth"
        else:
            gt_stem = stem
        out[gt_stem] = os.path.join(folder, f)
    return out

def load_npy(path):
    arr = np.load(path)
    arr = np.asarray(arr, dtype=np.float32)
    if arr.ndim==3 and arr.shape[-1]==1:
        arr = arr[...,0]
    return arr

def get_crop_box(h, w, dataset, crop_name):
    if crop_name == "none":
        return (0, h, 0, w)
    if dataset == "KITTI" and crop_name == "eigen":
        top = int(0.40810811 * h + 0.5)
        bottom = int(0.99189189 * h + 0.5)
        left = int(0.03594771 * w + 0.5)
        right = int(0.96405229 * w + 0.5)
        return (top, bottom, left, right)
    if dataset == "NYUv2" and crop_name == "eigen":
        top, bottom, left, right = 20, 460, 24, 616
        if h != 480 or w != 640:
            ty = int(top * h / 480); by = int(bottom * h / 480)
            lx = int(left * w / 640); rx = int(right * w / 640)
            return (ty, by, lx, rx)
        return (top, bottom, left, right)
    # DIODE：不裁剪
    return (0, h, 0, w)

def apply_cap(depth, cap):
    if cap and cap>0:
        depth = np.clip(depth, 0, float(cap))
    return depth

def align_depth(pred, gt, mask, mode="ls"):
    v = mask & np.isfinite(gt) & np.isfinite(pred)
    if v.sum()==0: return pred
    p = pred[v].astype(np.float64); g = gt[v].astype(np.float64)
    if mode=="none": return pred
    if mode=="scale":
        den = np.dot(p,p); s = (np.dot(g,p)/den) if den>0 else 1.0
        out = pred.copy(); out[v] = p*s; return out
    if mode=="ls":
        n = p.size
        A11 = np.dot(p,p); A12 = p.sum(); A22 = n
        b1  = np.dot(p,g); b2  = g.sum()
        det = A11*A22 - A12*A12
        if abs(det)<1e-12: s,t = 1.0,0.0
        else:
            s = ( b1*A22 - b2*A12)/det
            t = (-b1*A12 + b2*A11)/det
        out = pred.copy(); out[v] = p*s + t; return out
    return pred

def absrel_percent(pred, gt, mask, eps=1e-12, gt_min=1e-3):
    v = mask & (gt>gt_min) & np.isfinite(gt) & np.isfinite(pred)
    if v.sum()==0: return np.nan
    p = pred[v]; g = gt[v]
    return float(np.mean(np.abs(p-g)/(g+eps)))*100.0

def delta1_percent(pred, gt, mask, gt_min=1e-6, p_min=1e-6):
    v = mask & (gt>gt_min) & (pred>p_min) & np.isfinite(gt) & np.isfinite(pred)
    if v.sum()==0: return np.nan
    p = pred[v]; g = gt[v]
    r = np.maximum(p/g, g/p)
    return float(np.mean(r<1.25))*100.0

def evaluate_one(exp):
    """exp: dict 包含 pred_dir, gt_dir, mask_dir, dataset, crop, cap, alignment, pred_form, unit_div, save_prefix"""
    pred_idx = list_stems(exp["pred_dir"])
    gt_idx   = list_stems(exp["gt_dir"])
    mask_idx = list_mask_index(exp.get("mask_dir"))

    stems = sorted(set(pred_idx.keys()) & set(gt_idx.keys()))
    if len(stems)==0:
        print(f"[{exp['save_prefix']}] 配对样本为 0，请检查路径。")
        return None, None

    # 计算裁剪框
    tmp_gt = load_npy(gt_idx[stems[0]])
    crop_box = get_crop_box(tmp_gt.shape[0], tmp_gt.shape[1], exp["dataset"], exp["crop"])

    rows = []
    for stem in stems:
        try:
            pred = load_npy(pred_idx[stem])
            gt   = load_npy(gt_idx[stem])

            # 尺寸对齐
            if pred.shape != gt.shape:
                pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)

            # 掩码
            if exp.get("mask_dir") and stem in mask_idx:
                m = load_npy(mask_idx[stem]).astype(bool)
                if m.shape != gt.shape:
                    m = cv2.resize(m.astype(np.uint8), (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_NEAREST).astype(bool)
                mask = m & np.isfinite(gt) & (gt>GT_MIN_VALID)
            else:
                mask = np.isfinite(gt) & (gt>GT_MIN_VALID)

            # 裁剪
            t,b,l,r = crop_box
            pred = pred[t:b, l:r]; gt = gt[t:b, l:r]; mask = mask[t:b, l:r]

            # 单位 & 反深度
            if exp.get("unit_div", 1.0) != 1.0:
                pred = pred / float(exp["unit_div"])
            if exp.get("pred_form","id") == "inv":
                pred = 1.0 / np.maximum(pred, 1e-12)

            # cap
            pred = apply_cap(pred, exp["cap"])
            gt   = apply_cap(gt,   exp["cap"])

            # 对齐
            pred_aligned = align_depth(pred, gt, mask, exp["alignment"])

            # 指标
            a = absrel_percent(pred_aligned, gt, mask)
            d = delta1_percent(pred_aligned, gt, mask)
            rows.append({"stem": stem, "AbsRel(%)": a, "Delta1(%)": d})
        except Exception as e:
            rows.append({"stem": stem, "AbsRel(%)": np.nan, "Delta1(%)": np.nan})

    df = pd.DataFrame(rows).sort_values("stem")
    mean_a = float(df["AbsRel(%)"].mean(skipna=True))
    mean_d = float(df["Delta1(%)"].mean(skipna=True))
    return df, {"pairs": len(stems), "AbsRel(%)": mean_a, "Delta1(%)": mean_d}

# -------------------- 仅保留 4 组实验（DAv2/Marigold × DIODE） --------------------
OUT_DIR = "/kaggle/working/batch_output"
os.makedirs(OUT_DIR, exist_ok=True)

experiments = [
    # ===================== DepthAnythingV2 =====================
    # DIODE OUTDOOR
    {
        "save_prefix": "dav2_DIODE_OUTDOOR",
        "pred_dir": "/kaggle/input/test-results/selected_npys/outdoor/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/selected_npys/outdoor/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/outdoor",
        "dataset": "DIODE_OUTDOOR", "crop": "none", "cap": 300.0, "alignment": "ls",
        "pred_form": "auto", "unit_div": 1.0,
    },
    # DIODE INDOOR
    {
        "save_prefix": "dav2_DIODE_INDOOR",
        "pred_dir": "/kaggle/input/test-results/selected_npys/indoors/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/selected_npys/indoors/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/indoors",
        "dataset": "DIODE_INDOOR", "crop": "none", "cap": 50.0, "alignment": "ls",
        "pred_form": "auto", "unit_div": 1.0,
    },

    # ===================== Marigold =====================
    # DIODE OUTDOOR
    {
        "save_prefix": "marigold_DIODE_OUTDOOR",
        "pred_dir": "/kaggle/input/test-results/marigold_npys_only/outdoor/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/marigold_npys_only/outdoor/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/outdoor",
        "dataset": "DIODE_OUTDOOR", "crop": "none", "cap": 300.0, "alignment": "ls",
        "pred_form": "id", "unit_div": 1.0,
    },
    # DIODE INDOOR
    {
        "save_prefix": "marigold_DIODE_INDOOR",
        "pred_dir": "/kaggle/input/test-results/marigold_npys_only/indoors/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/marigold_npys_only/indoors/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/indoors",
        "dataset": "DIODE_INDOOR", "crop": "none", "cap": 50.0, "alignment": "ls",
        "pred_form": "id", "unit_div": 1.0,
    },
]

# -------------------- 执行评测 & 保存 --------------------
summary_rows = []
for exp in experiments:
    print(f"\n=== Evaluating: {exp['save_prefix']} ===")
    df, mean_stats = evaluate_one(exp)
    if df is None:
        continue
    csv_path = os.path.join(OUT_DIR, f"{exp['save_prefix']}_metrics.csv")
    df.to_csv(csv_path, index=False)
    print(f"Saved per-image metrics -> {csv_path}")
    print(f"Pairs={mean_stats['pairs']}  Mean AbsRel={mean_stats['AbsRel(%)']:.2f}%  Mean δ1={mean_stats['Delta1(%)']:.2f}%")

    summary_rows.append({
        "exp": exp["save_prefix"],
        "pairs": mean_stats["pairs"],
        "AbsRel_mean(%)": mean_stats["AbsRel(%)"],
        "Delta1_mean(%)": mean_stats["Delta1(%)"],
        "dataset": exp["dataset"],
        "crop": exp["crop"],
        "cap": exp["cap"],
        "alignment": exp["alignment"],
        "pred_form": exp["pred_form"],
        "unit_div": exp["unit_div"],
        "pred_dir": exp["pred_dir"],
        "gt_dir": exp["gt_dir"],
        "mask_dir": exp["mask_dir"],
    })

summary = pd.DataFrame(summary_rows).sort_values(["dataset","exp"])
summary_path = os.path.join(OUT_DIR, "summary.csv")
summary.to_csv(summary_path, index=False)
print("\n=== Summary ===")
print(summary.to_string(index=False))
print(f"\nSummary saved -> {summary_path}")



=== Evaluating: dav2_DIODE_OUTDOOR ===
Saved per-image metrics -> /kaggle/working/batch_output/dav2_DIODE_OUTDOOR_metrics.csv
Pairs=446  Mean AbsRel=84.61%  Mean δ1=58.59%

=== Evaluating: dav2_DIODE_INDOOR ===
Saved per-image metrics -> /kaggle/working/batch_output/dav2_DIODE_INDOOR_metrics.csv
Pairs=325  Mean AbsRel=57.37%  Mean δ1=67.25%

=== Evaluating: marigold_DIODE_OUTDOOR ===
Saved per-image metrics -> /kaggle/working/batch_output/marigold_DIODE_OUTDOOR_metrics.csv
Pairs=446  Mean AbsRel=74.65%  Mean δ1=68.33%

=== Evaluating: marigold_DIODE_INDOOR ===
Saved per-image metrics -> /kaggle/working/batch_output/marigold_DIODE_INDOOR_metrics.csv
Pairs=325  Mean AbsRel=10.71%  Mean δ1=89.73%

=== Summary ===
                   exp  pairs  AbsRel_mean(%)  Delta1_mean(%)       dataset crop   cap alignment pred_form  unit_div                                                           pred_dir                                                       gt_dir                         mask_dir
 

In [7]:
# ===================== Monocular Depth Batch Evaluator (with AUTO polarity) =====================
import os
import numpy as np
import pandas as pd
import cv2

# ---- optional SciPy Spearman; fallback if not available
try:
    from scipy.stats import spearmanr
    _HAVE_SCIPY = True
except Exception:
    _HAVE_SCIPY = False

GT_MIN_VALID = 1e-3   # guard tiny GT
EPS_INV = 1e-6        # for inverse transform stability

# -------------------- IO helpers --------------------
def list_stems(folder):
    files = [f for f in os.listdir(folder) if f.endswith(".npy")]
    return {os.path.splitext(f)[0]: os.path.join(folder, f) for f in files}

def list_mask_index(folder):
    """Map *_depth_mask.npy to GT stems (i.e., *_depth.npy) for pairing."""
    if not folder or (isinstance(folder, str) and not os.path.isdir(folder)):
        return {}
    out = {}
    for f in os.listdir(folder):
        if not f.endswith(".npy"):
            continue
        stem = os.path.splitext(f)[0]
        gt_stem = stem[:-len("_depth_mask")] + "_depth" if stem.endswith("_depth_mask") else stem
        out[gt_stem] = os.path.join(folder, f)
    return out

def load_npy(path):
    arr = np.load(path)
    arr = np.asarray(arr, dtype=np.float32)
    if arr.ndim == 3 and arr.shape[-1] == 1:
        arr = arr[..., 0]
    return arr

# -------------------- crops / caps / alignment --------------------
def get_crop_box(h, w, dataset, crop_name):
    if crop_name == "none":
        return (0, h, 0, w)
    if dataset == "KITTI" and crop_name == "eigen":
        top    = int(0.40810811 * h + 0.5)
        bottom = int(0.99189189 * h + 0.5)
        left   = int(0.03594771 * w + 0.5)
        right  = int(0.96405229 * w + 0.5)
        return (top, bottom, left, right)
    if dataset == "NYUv2" and crop_name == "eigen":
        top, bottom, left, right = 20, 460, 24, 616
        if (h, w) != (480, 640):
            ty = int(top * h / 480);  by = int(bottom * h / 480)
            lx = int(left * w / 640); rx = int(right * w / 640)
            return (ty, by, lx, rx)
        return (top, bottom, left, right)
    # default: no crop
    return (0, h, 0, w)

def apply_cap(depth, cap):
    if cap and cap > 0:
        depth = np.clip(depth, 0, float(cap))
    return depth

def align_depth(pred, gt, mask, mode="ls"):
    v = mask & np.isfinite(gt) & np.isfinite(pred)
    if v.sum() == 0:
        return pred
    p = pred[v].astype(np.float64); g = gt[v].astype(np.float64)

    if mode == "none":
        return pred

    if mode == "scale":
        den = np.dot(p, p)
        s = (np.dot(g, p) / den) if den > 0 else 1.0
        out = pred.copy()
        out[v] = p * s
        return out

    if mode == "ls":
        n = p.size
        A11 = np.dot(p, p); A12 = p.sum(); A22 = n
        b1  = np.dot(p, g); b2  = g.sum()
        det = A11 * A22 - A12 * A12
        if abs(det) < 1e-12:
            s, t = 1.0, 0.0
        else:
            s = ( b1 * A22 - b2 * A12) / det
            t = (-b1 * A12 + b2 * A11) / det
        out = pred.copy()
        out[v] = p * s + t
        return out

    return pred

# -------------------- metrics --------------------
def absrel_percent(pred, gt, mask, eps=1e-12, gt_min=1e-3):
    v = mask & (gt > gt_min) & np.isfinite(gt) & np.isfinite(pred)
    if v.sum() == 0: return np.nan
    p = pred[v]; g = gt[v]
    return float(np.mean(np.abs(p - g) / (g + eps))) * 100.0

def delta1_percent(pred, gt, mask, gt_min=1e-6, p_min=1e-6):
    v = mask & (gt > gt_min) & (pred > p_min) & np.isfinite(gt) & np.isfinite(pred)
    if v.sum() == 0: return np.nan
    p = pred[v]; g = gt[v]
    r = np.maximum(p / g, g / p)
    return float(np.mean(r < 1.25)) * 100.0

# -------------------- polarity (AUTO) --------------------
def _spearman_corr(a, b):
    if _HAVE_SCIPY:
        return float(spearmanr(a, b).correlation)
    # fallback: simple rank correlation
    ra = np.argsort(np.argsort(a))
    rb = np.argsort(np.argsort(b))
    va = (ra - ra.mean()) / (ra.std() + 1e-12)
    vb = (rb - rb.mean()) / (rb.std() + 1e-12)
    return float((va * vb).mean())

def pick_form_auto(pred, gt, mask):
    v = mask & np.isfinite(gt) & np.isfinite(pred) & (gt > 1e-3)
    if v.sum() < 100:
        return "id"  # not enough valid pixels
    p = pred[v].astype(np.float32)
    g = gt[v].astype(np.float32)
    rho_id  = _spearman_corr(p, g)
    rho_inv = _spearman_corr(1.0 / np.maximum(p, EPS_INV), g)
    return "inv" if rho_inv > rho_id else "id"

# -------------------- main evaluator --------------------
def evaluate_one(exp, out_dir):
    """
    exp keys:
      pred_dir, gt_dir, mask_dir or None,
      dataset, crop, cap, alignment ('none'|'scale'|'ls'),
      pred_form in {'id','inv','auto'}, unit_div, save_prefix
    """
    os.makedirs(out_dir, exist_ok=True)

    pred_idx = list_stems(exp["pred_dir"])
    gt_idx   = list_stems(exp["gt_dir"])
    mask_idx = list_mask_index(exp.get("mask_dir"))

    stems = sorted(set(pred_idx.keys()) & set(gt_idx.keys()))
    if len(stems) == 0:
        print(f"[{exp['save_prefix']}] 0 pairs — check paths.")
        return None, None

    # crop box
    tmp_gt = load_npy(gt_idx[stems[0]])
    t, b, l, r = get_crop_box(tmp_gt.shape[0], tmp_gt.shape[1], exp["dataset"], exp["crop"])

    rows = []
    auto_pick = {"id": 0, "inv": 0}

    for stem in stems:
        try:
            pred = load_npy(pred_idx[stem])
            gt   = load_npy(gt_idx[stem])

            # resize pred to GT
            if pred.shape != gt.shape:
                pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)

            # mask
            if exp.get("mask_dir") and stem in mask_idx:
                m = load_npy(mask_idx[stem]).astype(bool)
                if m.shape != gt.shape:
                    m = cv2.resize(m.astype(np.uint8), (gt.shape[1], gt.shape[0]),
                                   interpolation=cv2.INTER_NEAREST).astype(bool)
                mask = m & np.isfinite(gt) & (gt > GT_MIN_VALID)
            else:
                mask = np.isfinite(gt) & (gt > GT_MIN_VALID)

            # crop
            pred = pred[t:b, l:r]
            gt   = gt[t:b, l:r]
            mask = mask[t:b, l:r]

            # unit & polarity
            if exp.get("unit_div", 1.0) != 1.0:
                pred = pred / float(exp["unit_div"])

            pf = exp.get("pred_form", "id")
            if pf == "auto":
                pf = pick_form_auto(pred, gt, mask)
                auto_pick[pf] += 1
            if pf == "inv":
                pred = 1.0 / np.maximum(pred, EPS_INV)

            # cap
            pred = apply_cap(pred, exp["cap"])
            gt   = apply_cap(gt,   exp["cap"])

            # alignment
            pred_aligned = align_depth(pred, gt, mask, exp["alignment"])

            # metrics
            a = absrel_percent(pred_aligned, gt, mask)
            d = delta1_percent(pred_aligned, gt, mask)
            rows.append({"stem": stem, "AbsRel(%)": a, "Delta1(%)": d,
                         "picked_form": pf})
        except Exception:
            rows.append({"stem": stem, "AbsRel(%)": np.nan, "Delta1(%)": np.nan,
                         "picked_form": "err"})

    df = pd.DataFrame(rows).sort_values("stem")
    # save per-image
    csv_path = os.path.join(out_dir, f"{exp['save_prefix']}_metrics.csv")
    df.to_csv(csv_path, index=False)

    # summary
    mean_a = float(df["AbsRel(%)"].mean(skipna=True))
    med_a  = float(df["AbsRel(%)"].median(skipna=True))
    mean_d = float(df["Delta1(%)"].mean(skipna=True))
    med_d  = float(df["Delta1(%)"].median(skipna=True))

    print(f"=== {exp['save_prefix']} ===")
    if exp.get("pred_form") == "auto":
        print(f"Auto polarity picks -> id:{auto_pick['id']}  inv:{auto_pick['inv']}")
    print(f"Pairs={len(stems)}  Mean AbsRel={mean_a:.2f}%  Mean δ1={mean_d:.2f}%"
          f"  | Median AbsRel={med_a:.2f}%  Median δ1={med_d:.2f}%")
    print(f"Saved per-image metrics -> {csv_path}")

    return df, {
        "pairs": len(stems),
        "AbsRel_mean(%)": mean_a, "AbsRel_median(%)": med_a,
        "Delta1_mean(%)": mean_d, "Delta1_median(%)": med_d
    }

# ===================== Experiments (edit your paths here) =====================
OUT_DIR = "/kaggle/working/batch_output"
os.makedirs(OUT_DIR, exist_ok=True)

experiments = [
    # ---- DepthAnythingV2 × DIODE（建议：INDOOR=inv, OUTDOOR先auto） ----
    {
        "save_prefix": "dav2_DIODE_OUTDOOR",
        # 如果你有“修正 BGR 后”的新导出，请改成 /kaggle/working/download/outdoor/...
        "pred_dir": "/kaggle/input/test-results/selected_npys/outdoor/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/selected_npys/outdoor/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/outdoor",
        "dataset": "DIODE_OUTDOOR", "crop": "none", "cap": 300.0,
        "alignment": "ls", "pred_form": "auto", "unit_div": 1.0,
    },
    {
        "save_prefix": "dav2_DIODE_INDOOR",
        "pred_dir": "/kaggle/input/test-results/selected_npys/indoors/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/selected_npys/indoors/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/indoors",
        "dataset": "DIODE_INDOOR", "crop": "none", "cap": 50.0,
        "alignment": "ls", "pred_form": "inv",  # 室内多为 inverse-depth
        "unit_div": 1.0,
    },

    # ---- Marigold × DIODE（深度型 + LS 对齐） ----
    {
        "save_prefix": "marigold_DIODE_OUTDOOR",
        "pred_dir": "/kaggle/input/test-results/marigold_npys_only/outdoor/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/marigold_npys_only/outdoor/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/outdoor",
        "dataset": "DIODE_OUTDOOR", "crop": "none", "cap": 300.0,
        "alignment": "ls", "pred_form": "id", "unit_div": 1.0,
    },
    {
        "save_prefix": "marigold_DIODE_INDOOR",
        "pred_dir": "/kaggle/input/test-results/marigold_npys_only/indoors/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/marigold_npys_only/indoors/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/indoors",
        "dataset": "DIODE_INDOOR", "crop": "none", "cap": 50.0,
        "alignment": "ls", "pred_form": "id", "unit_div": 1.0,
    },
]

# ===================== Run =====================
summary_rows = []
for exp in experiments:
    _, stats = evaluate_one(exp, OUT_DIR)
    if stats:
        summary_rows.append({
            "exp": exp["save_prefix"],
            "pairs": stats["pairs"],
            "AbsRel_mean(%)": stats["AbsRel_mean(%)"],
            "AbsRel_median(%)": stats["AbsRel_median(%)"],
            "Delta1_mean(%)": stats["Delta1_mean(%)"],
            "Delta1_median(%)": stats["Delta1_median(%)"],
            "dataset": exp["dataset"],
            "crop": exp["crop"], "cap": exp["cap"],
            "alignment": exp["alignment"], "pred_form": exp["pred_form"],
            "unit_div": exp["unit_div"],
            "pred_dir": exp["pred_dir"], "gt_dir": exp["gt_dir"], "mask_dir": exp["mask_dir"],
        })

summary = pd.DataFrame(summary_rows).sort_values(["dataset","exp"])
summary_path = os.path.join(OUT_DIR, "summary.csv")
summary.to_csv(summary_path, index=False)
print("\n=== Summary ===")
print(summary.to_string(index=False))
print(f"\nSummary saved -> {summary_path}")


=== dav2_DIODE_OUTDOOR ===
Auto polarity picks -> id:9  inv:437
Pairs=446  Mean AbsRel=99.74%  Mean δ1=42.90%  | Median AbsRel=62.86%  Median δ1=37.68%
Saved per-image metrics -> /kaggle/working/batch_output/dav2_DIODE_OUTDOOR_metrics.csv
=== dav2_DIODE_INDOOR ===
Pairs=325  Mean AbsRel=23.55%  Mean δ1=73.06%  | Median AbsRel=15.97%  Median δ1=86.50%
Saved per-image metrics -> /kaggle/working/batch_output/dav2_DIODE_INDOOR_metrics.csv
=== marigold_DIODE_OUTDOOR ===
Pairs=446  Mean AbsRel=74.65%  Mean δ1=68.33%  | Median AbsRel=36.63%  Median δ1=75.04%
Saved per-image metrics -> /kaggle/working/batch_output/marigold_DIODE_OUTDOOR_metrics.csv
=== marigold_DIODE_INDOOR ===
Pairs=325  Mean AbsRel=10.71%  Mean δ1=89.73%  | Median AbsRel=6.90%  Median δ1=96.16%
Saved per-image metrics -> /kaggle/working/batch_output/marigold_DIODE_INDOOR_metrics.csv

=== Summary ===
                   exp  pairs  AbsRel_mean(%)  AbsRel_median(%)  Delta1_mean(%)  Delta1_median(%)       dataset crop   cap alig

In [4]:
import os, numpy as np, pandas as pd, cv2
from scipy.stats import spearmanr

GT_MIN_VALID = 1e-3

def list_stems(folder):
    files = [f for f in os.listdir(folder) if f.endswith(".npy")]
    return {os.path.splitext(f)[0]: os.path.join(folder, f) for f in files}

def list_mask_index(folder):
    if not folder or (isinstance(folder, str) and not os.path.isdir(folder)):
        return {}
    out = {}
    for f in os.listdir(folder):
        if not f.endswith(".npy"): continue
        stem = os.path.splitext(f)[0]
        gt_stem = stem[:-len("_depth_mask")] + "_depth" if stem.endswith("_depth_mask") else stem
        out[gt_stem] = os.path.join(folder, f)
    return out

def load_npy(path):
    arr = np.load(path).astype(np.float32)
    if arr.ndim==3 and arr.shape[-1]==1: arr = arr[...,0]
    return arr

def get_crop_box(h, w, dataset, crop_name):
    if crop_name == "none": return (0,h,0,w)
    if dataset=="KITTI" and crop_name=="eigen":
        top=int(0.40810811*h+0.5); bottom=int(0.99189189*h+0.5)
        left=int(0.03594771*w+0.5); right=int(0.96405229*w+0.5)
        return (top,bottom,left,right)
    if dataset=="NYUv2" and crop_name=="eigen":
        top,bottom,left,right = 20,460,24,616
        if (h,w)!=(480,640):
            ty=int(top*h/480); by=int(bottom*h/480)
            lx=int(left*w/640); rx=int(right*w/640)
            return (ty,by,lx,rx)
        return (top,bottom,left,right)
    return (0,h,0,w)

def apply_cap(x, cap):
    if cap and cap>0: x = np.clip(x, 0, float(cap))
    return x

def align_depth(pred, gt, mask, mode="ls"):
    v = mask & np.isfinite(gt) & np.isfinite(pred)
    if v.sum()==0: return pred
    p = pred[v].astype(np.float64); g = gt[v].astype(np.float64)
    if mode=="none": return pred
    if mode=="scale":
        den = np.dot(p,p); s = (np.dot(g,p)/den) if den>0 else 1.0
        out = pred.copy(); out[v] = p*s; return out
    if mode=="ls":
        n=p.size; A11=np.dot(p,p); A12=p.sum(); A22=n
        b1=np.dot(p,g); b2=g.sum(); det=A11*A22-A12*A12
        if abs(det)<1e-12: s,t = 1.0,0.0
        else:
            s = ( b1*A22 - b2*A12)/det
            t = (-b1*A12 + b2*A11)/det
        out = pred.copy(); out[v] = p*s + t; return out
    return pred

def absrel_percent(pred, gt, mask, eps=1e-12, gt_min=1e-3):
    v = mask & (gt>gt_min) & np.isfinite(gt) & np.isfinite(pred)
    if v.sum()==0: return np.nan
    p = pred[v]; g = gt[v]
    return float(np.mean(np.abs(p-g)/(g+eps)))*100.0

def delta1_percent(pred, gt, mask, gt_min=1e-6, p_min=1e-6):
    v = mask & (gt>gt_min) & (pred>p_min) & np.isfinite(gt) & np.isfinite(pred)
    if v.sum()==0: return np.nan
    p = pred[v]; g = gt[v]
    r = np.maximum(p/g, g/p)
    return float(np.mean(r<1.25))*100.0

def pick_form_auto(pred, gt, mask):
    v = mask & np.isfinite(gt) & np.isfinite(pred) & (gt>1e-3)
    if v.sum()<100: return "id"
    p,g = pred[v], gt[v]
    rho_id  = spearmanr(p, g).correlation
    rho_inv = spearmanr(1.0/np.maximum(p,1e-6), g).correlation
    return "inv" if (rho_inv>rho_id) else "id"

def safe_clip_rate(arr, cap, mask=None):
    if not cap or cap<=0: return np.nan
    if mask is None: mask = np.ones_like(arr, dtype=bool)
    v = mask & np.isfinite(arr)
    if v.sum()==0: return np.nan
    return float(np.mean(arr[v] >= (cap - 1e-3)))

def evaluate_debug(exp, diag_dir):
    os.makedirs(diag_dir, exist_ok=True)

    pred_idx = list_stems(exp["pred_dir"])
    gt_idx   = list_stems(exp["gt_dir"])
    mask_idx = list_mask_index(exp.get("mask_dir"))

    stems = sorted(set(pred_idx.keys()) & set(gt_idx.keys()))
    if not stems:
        print(f"[{exp['save_prefix']}] 0 pairs; check paths.")
        return None, None

    # crop box
    tmp = load_npy(gt_idx[stems[0]])
    t,b,l,r = get_crop_box(tmp.shape[0], tmp.shape[1], exp["dataset"], exp["crop"])

    rows = []
    for stem in stems:
        try:
            pred = load_npy(pred_idx[stem])
            gt   = load_npy(gt_idx[stem])

            if pred.shape != gt.shape:
                pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)

            if exp.get("mask_dir") and stem in mask_idx:
                m = load_npy(mask_idx[stem]).astype(bool)
                if m.shape != gt.shape:
                    m = cv2.resize(m.astype(np.uint8), (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_NEAREST).astype(bool)
                mask = m & np.isfinite(gt) & (gt>GT_MIN_VALID)
            else:
                mask = np.isfinite(gt) & (gt>GT_MIN_VALID)

            # crop
            pred = pred[t:b, l:r]; gt = gt[t:b, l:r]; mask = mask[t:b, l:r]

            # unit & polarity
            if exp.get("unit_div",1.0) != 1.0:
                pred = pred / float(exp["unit_div"])
            pf = exp.get("pred_form","id")
            if pf == "auto":
                pf = pick_form_auto(pred, gt, mask)
            if pf == "inv":
                pred = 1.0/np.maximum(pred, 1e-6)

            # cap
            cap = exp["cap"]
            pred_capped = apply_cap(pred, cap)
            gt_capped   = apply_cap(gt,   cap)

            # clip rate BEFORE alignment
            clip_b = safe_clip_rate(pred_capped, cap, mask)

            # align
            pred_aligned = align_depth(pred_capped, gt_capped, mask, exp["alignment"])

            # OPTIONAL: 若想看对齐后是否又越界，可再 cap 一次再算
            clip_a = safe_clip_rate(pred_aligned, cap, mask)

            # metrics
            a = absrel_percent(pred_aligned, gt_capped, mask)
            d = delta1_percent(pred_aligned, gt_capped, mask)

            rows.append({
                "stem": stem,
                "AbsRel(%)": a,
                "Delta1(%)": d,
                "clip_rate_before": clip_b,
                "clip_rate_after": clip_a,
                "valid_ratio": float(np.mean(mask)),
                "picked_form": pf,
            })
        except Exception as e:
            rows.append({"stem": stem, "AbsRel(%)": np.nan, "Delta1(%)": np.nan,
                         "clip_rate_before": np.nan, "clip_rate_after": np.nan,
                         "valid_ratio": np.nan, "picked_form": "err"})

    df = pd.DataFrame(rows).sort_values("stem")

    # aggregate: mean & median
    summary = {
        "pairs": len(stems),
        "AbsRel_mean(%)": float(df["AbsRel(%)"].mean(skipna=True)),
        "AbsRel_median(%)": float(df["AbsRel(%)"].median(skipna=True)),
        "Delta1_mean(%)": float(df["Delta1(%)"].mean(skipna=True)),
        "Delta1_median(%)": float(df["Delta1(%)"].median(skipna=True)),
        "clip_before_mean": float(df["clip_rate_before"].mean(skipna=True)),
        "clip_before_median": float(df["clip_rate_before"].median(skipna=True)),
        "clip_after_mean": float(df["clip_rate_after"].mean(skipna=True)),
        "clip_after_median": float(df["clip_rate_after"].median(skipna=True)),
    }

    # save
    df_path = os.path.join(diag_dir, f"{exp['save_prefix']}_diag.csv")
    df.to_csv(df_path, index=False)
    print(f"[{exp['save_prefix']}] pairs={summary['pairs']}"
          f" | AbsRel mean/med={summary['AbsRel_mean(%)']:.2f}/{summary['AbsRel_median(%)']:.2f}"
          f" | δ1 mean/med={summary['Delta1_mean(%)']:.2f}/{summary['Delta1_median(%)']:.2f}"
          f" | clip_before mean/med={summary['clip_before_mean']:.3f}/{summary['clip_before_median']:.3f}"
          f" | clip_after mean/med={summary['clip_after_mean']:.3f}/{summary['clip_after_median']:.3f}"
         )
    print(f"Saved per-image diagnostics -> {df_path}")

    # Top-K 观察：最差 AbsRel、最高截顶率
    K=5
    bad_absrel = df.sort_values("AbsRel(%)", ascending=False).head(K)[["stem","AbsRel(%)","Delta1(%)","clip_rate_before","clip_rate_after","picked_form"]]
    high_clip  = df.sort_values("clip_rate_before", ascending=False).head(K)[["stem","AbsRel(%)","Delta1(%)","clip_rate_before","clip_rate_after","picked_form"]]
    print("\nTop-5 worst AbsRel:\n", bad_absrel.to_string(index=False))
    print("\nTop-5 highest clip_rate_before:\n", high_clip.to_string(index=False))

    return df, summary

# ===== 示例：把你的实验配置丢进来即可（示意） =====
OUT_DIR = "/kaggle/working/diag"
os.makedirs(OUT_DIR, exist_ok=True)

experiments = [
    # 例：DAv2 DIODE OUTDOOR
    {
        "save_prefix": "dav2_DIODE_OUTDOOR",
        "pred_dir": "/kaggle/input/test-results/selected_npys/outdoor/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/selected_npys/outdoor/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/outdoor",
        "dataset": "DIODE_OUTDOOR", "crop": "none", "cap": 300.0,
        "alignment": "ls", "pred_form": "inv", "unit_div": 1.0,
    },
    # 例：Marigold DIODE OUTDOOR
    {
        "save_prefix": "marigold_DIODE_OUTDOOR",
        "pred_dir": "/kaggle/input/test-results/marigold_npys_only/outdoor/pred_raw_npy",
        "gt_dir":   "/kaggle/input/test-results/marigold_npys_only/outdoor/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/outdoor",
        "dataset": "DIODE_OUTDOOR", "crop": "none", "cap": 300.0,
        "alignment": "ls", "pred_form": "id", "unit_div": 1.0,
    },
]

for exp in experiments:
    evaluate_debug(exp, OUT_DIR)


[dav2_DIODE_OUTDOOR] pairs=446 | AbsRel mean/med=99.86/62.86 | δ1 mean/med=42.84/37.68 | clip_before mean/med=0.006/0.001 | clip_after mean/med=0.000/0.000
Saved per-image diagnostics -> /kaggle/working/diag/dav2_DIODE_OUTDOOR_diag.csv

Top-5 worst AbsRel:
                                                stem  AbsRel(%)  Delta1(%)  clip_rate_before  clip_rate_after picked_form
scene_00022_scan_00194__00022_00194_outdoor_180_030 731.794596  10.773680          0.042334              0.0         inv
scene_00023_scan_00200__00023_00200_outdoor_210_030 566.116381   5.572905          0.043884              0.0         inv
scene_00023_scan_00200__00023_00200_outdoor_190_030 505.341911   5.293456          0.047754              0.0         inv
scene_00022_scan_00194__00022_00194_outdoor_160_030 489.514732  16.350588          0.029346              0.0         inv
scene_00022_scan_00197__00022_00197_outdoor_060_020 489.243221  15.479204          0.057354              0.0         inv

Top-5 highest c

In [ ]:
# ===================== Batch evaluator (with inverse-depth support) =====================
import os, json
import numpy as np
import pandas as pd
import cv2

# -------------------- 公共函数 --------------------
GT_MIN_VALID = 1e-3  # 防止极小正数导致 AbsRel 爆炸

def list_stems(folder):
    files = [f for f in os.listdir(folder) if f.endswith(".npy")]
    return {os.path.splitext(f)[0]: os.path.join(folder, f) for f in files}

def list_mask_index(folder):
    """把 *_depth_mask.npy 映射到 GT 的 *_depth.npy 的 stem，便于和 GT 配对。"""
    if not folder or (isinstance(folder, str) and not os.path.isdir(folder)):
        return {}
    out = {}
    for f in os.listdir(folder):
        if not f.endswith(".npy"):
            continue
        stem = os.path.splitext(f)[0]
        if stem.endswith("_depth_mask"):
            gt_stem = stem[:-len("_depth_mask")] + "_depth"
        else:
            gt_stem = stem
        out[gt_stem] = os.path.join(folder, f)
    return out

def load_npy(path):
    arr = np.load(path)
    arr = np.asarray(arr, dtype=np.float32)
    if arr.ndim==3 and arr.shape[-1]==1:
        arr = arr[...,0]
    return arr

def get_crop_box(h, w, dataset, crop_name):
    if crop_name == "none":
        return (0, h, 0, w)
    if dataset == "KITTI" and crop_name == "eigen":
        top = int(0.40810811 * h + 0.5)
        bottom = int(0.99189189 * h + 0.5)
        left = int(0.03594771 * w + 0.5)
        right = int(0.96405229 * w + 0.5)
        return (top, bottom, left, right)
    if dataset == "NYUv2" and crop_name == "eigen":
        top, bottom, left, right = 20, 460, 24, 616
        if h != 480 or w != 640:
            ty = int(top * h / 480); by = int(bottom * h / 480)
            lx = int(left * w / 640); rx = int(right * w / 640)
            return (ty, by, lx, rx)
        return (top, bottom, left, right)
    # DIODE：不裁剪
    return (0, h, 0, w)

def apply_cap(depth, cap):
    if cap and cap>0:
        depth = np.clip(depth, 0, float(cap))
    return depth

def align_depth(pred, gt, mask, mode="ls"):
    v = mask & np.isfinite(gt) & np.isfinite(pred)
    if v.sum()==0: return pred
    p = pred[v].astype(np.float64); g = gt[v].astype(np.float64)
    if mode=="none": return pred
    if mode=="scale":
        den = np.dot(p,p); s = (np.dot(g,p)/den) if den>0 else 1.0
        out = pred.copy(); out[v] = p*s; return out
    if mode=="ls":
        n = p.size
        A11 = np.dot(p,p); A12 = p.sum(); A22 = n
        b1  = np.dot(p,g); b2  = g.sum()
        det = A11*A22 - A12*A12
        if abs(det)<1e-12: s,t = 1.0,0.0
        else:
            s = ( b1*A22 - b2*A12)/det
            t = (-b1*A12 + b2*A11)/det
        out = pred.copy(); out[v] = p*s + t; return out
    return pred

def absrel_percent(pred, gt, mask, eps=1e-12, gt_min=1e-3):
    v = mask & (gt>gt_min) & np.isfinite(gt) & np.isfinite(pred)
    if v.sum()==0: return np.nan
    p = pred[v]; g = gt[v]
    return float(np.mean(np.abs(p-g)/(g+eps)))*100.0

def delta1_percent(pred, gt, mask, gt_min=1e-6, p_min=1e-6):
    v = mask & (gt>gt_min) & (pred>p_min) & np.isfinite(gt) & np.isfinite(pred)
    if v.sum()==0: return np.nan
    p = pred[v]; g = gt[v]
    r = np.maximum(p/g, g/p)
    return float(np.mean(r<1.25))*100.0

def evaluate_one(exp):
    """exp: dict 包含 pred_dir, gt_dir, mask_dir, dataset, crop, cap, alignment, pred_form, unit_div, save_prefix"""
    pred_idx = list_stems(exp["pred_dir"])
    gt_idx   = list_stems(exp["gt_dir"])
    mask_idx = list_mask_index(exp.get("mask_dir"))

    stems = sorted(set(pred_idx.keys()) & set(gt_idx.keys()))
    if len(stems)==0:
        print(f"[{exp['save_prefix']}] 配对样本为 0，请检查路径。")
        return None, None

    # 计算裁剪框
    tmp_gt = load_npy(gt_idx[stems[0]])
    crop_box = get_crop_box(tmp_gt.shape[0], tmp_gt.shape[1], exp["dataset"], exp["crop"])

    rows = []
    for stem in stems:
        try:
            pred = load_npy(pred_idx[stem])
            gt   = load_npy(gt_idx[stem])

            # 尺寸对齐
            if pred.shape != gt.shape:
                pred = cv2.resize(pred, (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_LINEAR)

            # 掩码
            if exp.get("mask_dir") and stem in mask_idx:
                m = load_npy(mask_idx[stem]).astype(bool)
                if m.shape != gt.shape:
                    m = cv2.resize(m.astype(np.uint8), (gt.shape[1], gt.shape[0]), interpolation=cv2.INTER_NEAREST).astype(bool)
                mask = m & np.isfinite(gt) & (gt>GT_MIN_VALID)
            else:
                mask = np.isfinite(gt) & (gt>GT_MIN_VALID)

            # 裁剪
            t,b,l,r = crop_box
            pred = pred[t:b, l:r]; gt = gt[t:b, l:r]; mask = mask[t:b, l:r]

            # 单位 & 反深度
            if exp.get("unit_div", 1.0) != 1.0:
                pred = pred / float(exp["unit_div"])
            if exp.get("pred_form","id") == "inv":
                pred = 1.0 / np.maximum(pred, 1e-12)

            # cap
            pred = apply_cap(pred, exp["cap"])
            gt   = apply_cap(gt,   exp["cap"])

            # 对齐
            pred_aligned = align_depth(pred, gt, mask, exp["alignment"])

            # 指标
            a = absrel_percent(pred_aligned, gt, mask)
            d = delta1_percent(pred_aligned, gt, mask)
            rows.append({"stem": stem, "AbsRel(%)": a, "Delta1(%)": d})
        except Exception as e:
            rows.append({"stem": stem, "AbsRel(%)": np.nan, "Delta1(%)": np.nan})

    df = pd.DataFrame(rows).sort_values("stem")
    mean_a = float(df["AbsRel(%)"].mean(skipna=True))
    mean_d = float(df["Delta1(%)"].mean(skipna=True))
    return df, {"pairs": len(stems), "AbsRel(%)": mean_a, "Delta1(%)": mean_d}

# -------------------- 12 组实验配置（DAv2 / Marigold / ZoeDepth × DIODE/KITTI/NYUv2） --------------------
OUT_DIR = "/kaggle/working/batch_output"
os.makedirs(OUT_DIR, exist_ok=True)

experiments = [
    # ===================== DepthAnythingV2 =====================
    # DIODE OUTDOOR
    {
        "save_prefix": "dav2_DIODE_OUTDOOR",
        "pred_dir": "/kaggle/input/results/dav2_diode_preds_and_gt/dav2_diode_preds_and_gt/outdoor/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/dav2_diode_preds_and_gt/dav2_diode_preds_and_gt/outdoor/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/outdoor",
        "dataset": "DIODE_OUTDOOR", "crop": "none", "cap": 300.0, "alignment": "ls",
        "pred_form": "inv", "unit_div": 1.0,
    },
    # DIODE INDOOR
    {
        "save_prefix": "dav2_DIODE_INDOOR",
        "pred_dir": "/kaggle/input/results/dav2_diode_preds_and_gt/dav2_diode_preds_and_gt/indoors/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/dav2_diode_preds_and_gt/dav2_diode_preds_and_gt/indoors/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/indoors",
        "dataset": "DIODE_INDOOR", "crop": "none", "cap": 50.0, "alignment": "ls",
        "pred_form": "inv", "unit_div": 1.0,
    },
    # KITTI
    {
        "save_prefix": "dav2_KITTI",
        "pred_dir": "/kaggle/input/results/dav2_kitti_preds_and_gt/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/dav2_kitti_preds_and_gt/gt_npy",
        "mask_dir": None,
        "dataset": "KITTI", "crop": "eigen", "cap": 80.0, "alignment": "ls",
        "pred_form": "inv", "unit_div": 1.0,
    },
    # NYUv2
    {
        "save_prefix": "dav2_NYUv2",
        "pred_dir": "/kaggle/input/results/dav2_nyuv2_preds_and_gt/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/dav2_nyuv2_preds_and_gt/gt_npy",
        "mask_dir": None,
        "dataset": "NYUv2", "crop": "eigen", "cap": 10.0, "alignment": "ls",
        "pred_form": "inv", "unit_div": 1.0,
    },

    # ===================== Marigold =====================
    # DIODE OUTDOOR
    {
        "save_prefix": "marigold_DIODE_OUTDOOR",
        "pred_dir": "/kaggle/input/results/marigold_diode_preds_and_gt/marigold_diode_preds_and_gt/outdoor/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/marigold_diode_preds_and_gt/marigold_diode_preds_and_gt/outdoor/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/outdoor",
        "dataset": "DIODE_OUTDOOR", "crop": "none", "cap": 300.0, "alignment": "ls",
        "pred_form": "inv",  # 若结果异常，可切到 "id" 再试
        "unit_div": 1.0,
    },
    # DIODE INDOOR
    {
        "save_prefix": "marigold_DIODE_INDOOR",
        "pred_dir": "/kaggle/input/results/marigold_diode_preds_and_gt/marigold_diode_preds_and_gt/indoors/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/marigold_diode_preds_and_gt/marigold_diode_preds_and_gt/indoors/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/indoors",
        "dataset": "DIODE_INDOOR", "crop": "none", "cap": 50.0, "alignment": "ls",
        "pred_form": "inv",  # 若结果异常，可切到 "id" 再试
        "unit_div": 1.0,
    },
    # KITTI
    {
        "save_prefix": "marigold_KITTI",
        "pred_dir": "/kaggle/input/results/marigold_kitti_preds_and_gt/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/marigold_kitti_preds_and_gt/gt_npy",
        "mask_dir": None,
        "dataset": "KITTI", "crop": "eigen", "cap": 80.0, "alignment": "ls",
        "pred_form": "inv",  # 若结果异常，可切到 "id"
        "unit_div": 1.0,
    },
    # NYUv2
    {
        "save_prefix": "marigold_NYUv2",
        "pred_dir": "/kaggle/input/results/marigold_nyuv2_preds_and_gt/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/marigold_nyuv2_preds_and_gt/gt_npy",
        "mask_dir": None,
        "dataset": "NYUv2", "crop": "eigen", "cap": 10.0, "alignment": "ls",
        "pred_form": "inv",  # 若结果异常，可切到 "id"
        "unit_div": 1.0,
    },

    # ===================== ZoeDepth =====================
    # DIODE OUTDOOR
    {
        "save_prefix": "zoe_DIODE_OUTDOOR",
        "pred_dir": "/kaggle/input/results/zoe_diode_preds_and_gt/zoe_diode_preds_and_gt/outdoor/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/zoe_diode_preds_and_gt/zoe_diode_preds_and_gt/outdoor/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/outdoor",
        "dataset": "DIODE_OUTDOOR", "crop": "none", "cap": 300.0, "alignment": "ls",
        "pred_form": "id",   # ZoeDepth 常见是 metric depth；若异常再试 "inv"
        "unit_div": 1.0,
    },
    # DIODE INDOOR
    {
        "save_prefix": "zoe_DIODE_INDOOR",
        "pred_dir": "/kaggle/input/results/zoe_diode_preds_and_gt/zoe_diode_preds_and_gt/indoors/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/zoe_diode_preds_and_gt/zoe_diode_preds_and_gt/indoors/gt_npy",
        "mask_dir": "/kaggle/input/diode-mask/indoors",
        "dataset": "DIODE_INDOOR", "crop": "none", "cap": 50.0, "alignment": "ls",
        "pred_form": "id",   # 若异常再试 "inv"
        "unit_div": 1.0,
    },
    # KITTI
    {
        "save_prefix": "zoe_KITTI",
        "pred_dir": "/kaggle/input/results/zoe_kitti_preds_and_gt/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/zoe_kitti_preds_and_gt/gt_npy",
        "mask_dir": None,
        "dataset": "KITTI", "crop": "eigen", "cap": 80.0, "alignment": "ls",
        "pred_form": "id",   # 若异常再试 "inv"
        "unit_div": 1.0,
    },
    # NYUv2
    {
        "save_prefix": "zoe_NYUv2",
        "pred_dir": "/kaggle/input/results/zoe_nyuv2_preds_and_gt/pred_raw_npy",
        "gt_dir":   "/kaggle/input/results/zoe_nyuv2_preds_and_gt/gt_npy",
        "mask_dir": None,
        "dataset": "NYUv2", "crop": "eigen", "cap": 10.0, "alignment": "ls",
        "pred_form": "id",   # 若异常再试 "inv"
        "unit_div": 1.0,
    },
]

# -------------------- 执行评测 & 保存 --------------------
summary_rows = []
for exp in experiments:
    print(f"\n=== Evaluating: {exp['save_prefix']} ===")
    df, mean_stats = evaluate_one(exp)
    if df is None:
        continue
    csv_path = os.path.join(OUT_DIR, f"{exp['save_prefix']}_metrics.csv")
    df.to_csv(csv_path, index=False)
    print(f"Saved per-image metrics -> {csv_path}")
    print(f"Pairs={mean_stats['pairs']}  Mean AbsRel={mean_stats['AbsRel(%)']:.2f}%  Mean δ1={mean_stats['Delta1(%)']:.2f}%")

    summary_rows.append({
        "exp": exp["save_prefix"],
        "pairs": mean_stats["pairs"],
        "AbsRel_mean(%)": mean_stats["AbsRel(%)"],
        "Delta1_mean(%)": mean_stats["Delta1(%)"],
        "dataset": exp["dataset"],
        "crop": exp["crop"],
        "cap": exp["cap"],
        "alignment": exp["alignment"],
        "pred_form": exp["pred_form"],
        "unit_div": exp["unit_div"],
        "pred_dir": exp["pred_dir"],
        "gt_dir": exp["gt_dir"],
        "mask_dir": exp["mask_dir"],
    })

summary = pd.DataFrame(summary_rows).sort_values(["dataset","exp"])
summary_path = os.path.join(OUT_DIR, "summary.csv")
summary.to_csv(summary_path, index=False)
print("\n=== Summary ===")
print(summary.to_string(index=False))
print(f"\nSummary saved -> {summary_path}")
